# Summary:
- No SMOTE
- Running on the new vectors (window=8, word_count =1, and iter =50)
- Results:


    - SVM 
                     precision    recall  f1-score   support
                     
        0.0               0.92      0.75      0.83      2626
        1.0               0.75      0.92      0.83      2152
        avg / total       0.85      0.83      0.83      4778

        Confusion Matrix
            1979  647
            164  1988

    - DNN + SVM
    
                     precision    recall  f1-score   support

        0.0               0.87      0.84      0.86      2626
        1.0               0.82      0.85      0.83      2152
        avg / total       0.85      0.85      0.85      4778

        Confusion Matrix
            2214  412
            323  1829
               
    - Deeper DNN with dropout + SVM

                     precision    recall  f1-score   support

        0.0               0.87      0.83      0.85      2626
        1.0               0.80      0.85      0.83      2152
        avg / total       0.84      0.84      0.84      4778
    
        Confusion Matrix
            2181  445
             321 1831
    
    - Inception Batch Normalization + SVM

                     precision    recall  f1-score   support

        0.0               0.85      0.80      0.82      2626
        1.0               0.77      0.82      0.80      2152
        avg / total       0.81      0.81      0.81      4778

        Confusion Matrix
            2100  526
             382 1770

In [1]:
import glob
import multiprocessing
import time
import csv
import pickle
import os
import pandas as pd
import shutil
import numpy as np
import warnings
import tqdm
import logging
import mxnet as mx
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import basename
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/mxnet/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Populating the interactive namespace from numpy and matplotlib


In [2]:
# 2d to 4d conversion
def to4d(img):
    return img.reshape(img.shape[0], 1, 32, 32).astype(np.float32)

In [3]:
def modelMetric(mod, info, nd=True):
    if nd == True: 
        prob = mod.predict(val_iters).asnumpy()
    else:
        prob = mod.predict(val_iters)
    logging.info(info)
    # Check the accuracy from prediction
    val_iters.reset()

    y_batch = []
    for dbatch in val_iters:
        label = dbatch.label[0].asnumpy()
        pad = val_iters.getpad()
        real_size = label.shape[0] - pad
        y_batch.append(label[0:real_size])
    y = np.concatenate(y_batch)

    # get prediction label 
    py = np.argmax(prob, axis=1)
    acc1 = sum(py == y) / len(y)
    logging.info('Test accuracy = %f', acc1)
    cfn_matrix=confusion_matrix(y,py)
    print(classification_report (y,py))
    print(cfn_matrix)

# 1. Data Preparation

In [4]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [5]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [6]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [7]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [8]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((14333, 1024), (14333,), (4778, 1024), (4778,), 19111)

In [9]:
# creating 4d training/testing dataset
batch_size = 64
train_iters = mx.io.NDArrayIter(to4d(X_train_array), Y_train_array, batch_size, shuffle=True,label_name='svm_label')
val_iters = mx.io.NDArrayIter(to4d(X_test_array), Y_test_array, batch_size,label_name='svm_label')

# 2. Train models
## 2.1 SVM

In [10]:
# Single SVM
data = mx.symbol.Variable('data')
svm  = mx.symbol.FullyConnected(data = data, name='fc', num_hidden=2)

# Here we add the ultimate layer based on L2-SVM objective
ssvm = mx.symbol.SVMOutput(data=svm, name='svm')
# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart.format = 'png'
network_chart.render('SVM')

'SVM.png'

<img src='SVM.png'>

In [11]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

mod = mx.model.FeedForward(
    ctx = mx.gpu(0),     # use GPU 0 for training, others are same as before
    symbol = ssvm,       # network structure
    num_epoch = 300,     # number of data passes for training 
    learning_rate = 0.01 # learning rate of SGD 
)
mod.fit(
    X=train_iters,       # training data
    eval_data=val_iters, # validation data
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size) # output progress for each 200 data batches
)


INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [64]	Speed: 58444.27 samples/sec	accuracy=0.743652
INFO:root:Epoch[0] Batch [128]	Speed: 62076.54 samples/sec	accuracy=0.794678
INFO:root:Epoch[0] Batch [192]	Speed: 64086.77 samples/sec	accuracy=0.811035
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.244
INFO:root:Epoch[0] Validation-accuracy=0.832083
INFO:root:Epoch[1] Batch [64]	Speed: 70802.19 samples/sec	accuracy=0.849121
INFO:root:Epoch[1] Batch [128]	Speed: 71129.63 samples/sec	accuracy=0.826172
INFO:root:Epoch[1] Batch [192]	Speed: 61303.25 samples/sec	accuracy=0.835938
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.223
INFO:root:Epoch[1] Validation-accuracy=0.833333
INFO:root:Epoch[2] Batch [64]	Speed: 62046.72 samples/sec	accuracy=0.853271
INFO:root:Epoch[2] Batch [128]	Speed: 63994.86 samples/sec	accuracy=0.830566
INFO:root:Epoch[2] Batch [192]	Speed: 62204.88 samples/sec	accuracy=0.838135
INFO:root:Epoch[2] 

INFO:root:Epoch[22] Time cost=0.210
INFO:root:Epoch[22] Validation-accuracy=0.832708
INFO:root:Epoch[23] Batch [64]	Speed: 64218.53 samples/sec	accuracy=0.852295
INFO:root:Epoch[23] Batch [128]	Speed: 62509.76 samples/sec	accuracy=0.836914
INFO:root:Epoch[23] Batch [192]	Speed: 63708.66 samples/sec	accuracy=0.840820
INFO:root:Epoch[23] Resetting Data Iterator
INFO:root:Epoch[23] Time cost=0.234
INFO:root:Epoch[23] Validation-accuracy=0.832708
INFO:root:Epoch[24] Batch [64]	Speed: 61586.97 samples/sec	accuracy=0.852295
INFO:root:Epoch[24] Batch [128]	Speed: 70552.43 samples/sec	accuracy=0.836914
INFO:root:Epoch[24] Batch [192]	Speed: 65962.00 samples/sec	accuracy=0.841064
INFO:root:Epoch[24] Resetting Data Iterator
INFO:root:Epoch[24] Time cost=0.232
INFO:root:Epoch[24] Validation-accuracy=0.832708
INFO:root:Epoch[25] Batch [64]	Speed: 67131.42 samples/sec	accuracy=0.852295
INFO:root:Epoch[25] Batch [128]	Speed: 66596.90 samples/sec	accuracy=0.836914
INFO:root:Epoch[25] Batch [192]	Spee

INFO:root:Epoch[45] Batch [192]	Speed: 61464.24 samples/sec	accuracy=0.842773
INFO:root:Epoch[45] Resetting Data Iterator
INFO:root:Epoch[45] Time cost=0.236
INFO:root:Epoch[45] Validation-accuracy=0.831042
INFO:root:Epoch[46] Batch [64]	Speed: 65955.16 samples/sec	accuracy=0.851318
INFO:root:Epoch[46] Batch [128]	Speed: 61917.02 samples/sec	accuracy=0.836670
INFO:root:Epoch[46] Batch [192]	Speed: 64929.42 samples/sec	accuracy=0.842529
INFO:root:Epoch[46] Resetting Data Iterator
INFO:root:Epoch[46] Time cost=0.229
INFO:root:Epoch[46] Validation-accuracy=0.831042
INFO:root:Epoch[47] Batch [64]	Speed: 72840.19 samples/sec	accuracy=0.851318
INFO:root:Epoch[47] Batch [128]	Speed: 77087.13 samples/sec	accuracy=0.836914
INFO:root:Epoch[47] Batch [192]	Speed: 72481.56 samples/sec	accuracy=0.842773
INFO:root:Epoch[47] Resetting Data Iterator
INFO:root:Epoch[47] Time cost=0.203
INFO:root:Epoch[47] Validation-accuracy=0.831042
INFO:root:Epoch[48] Batch [64]	Speed: 67857.42 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [128]	Speed: 70620.58 samples/sec	accuracy=0.837646
INFO:root:Epoch[68] Batch [192]	Speed: 67814.29 samples/sec	accuracy=0.842529
INFO:root:Epoch[68] Resetting Data Iterator
INFO:root:Epoch[68] Time cost=0.216
INFO:root:Epoch[68] Validation-accuracy=0.830625
INFO:root:Epoch[69] Batch [64]	Speed: 67239.40 samples/sec	accuracy=0.852295
INFO:root:Epoch[69] Batch [128]	Speed: 68147.58 samples/sec	accuracy=0.837646
INFO:root:Epoch[69] Batch [192]	Speed: 69416.42 samples/sec	accuracy=0.842529
INFO:root:Epoch[69] Resetting Data Iterator
INFO:root:Epoch[69] Time cost=0.216
INFO:root:Epoch[69] Validation-accuracy=0.830625
INFO:root:Epoch[70] Batch [64]	Speed: 61090.06 samples/sec	accuracy=0.852295
INFO:root:Epoch[70] Batch [128]	Speed: 66298.52 samples/sec	accuracy=0.837646
INFO:root:Epoch[70] Batch [192]	Speed: 63108.38 samples/sec	accuracy=0.842529
INFO:root:Epoch[70] Resetting Data Iterator
INFO:root:Epoch[70] Time cost=0.229
INFO:root:Epoch[70] Validation-accuracy=

INFO:root:Epoch[90] Validation-accuracy=0.831042
INFO:root:Epoch[91] Batch [64]	Speed: 67538.63 samples/sec	accuracy=0.852295
INFO:root:Epoch[91] Batch [128]	Speed: 62748.34 samples/sec	accuracy=0.837646
INFO:root:Epoch[91] Batch [192]	Speed: 70726.99 samples/sec	accuracy=0.843018
INFO:root:Epoch[91] Resetting Data Iterator
INFO:root:Epoch[91] Time cost=0.219
INFO:root:Epoch[91] Validation-accuracy=0.831042
INFO:root:Epoch[92] Batch [64]	Speed: 69423.43 samples/sec	accuracy=0.852295
INFO:root:Epoch[92] Batch [128]	Speed: 65599.06 samples/sec	accuracy=0.837646
INFO:root:Epoch[92] Batch [192]	Speed: 65183.16 samples/sec	accuracy=0.843018
INFO:root:Epoch[92] Resetting Data Iterator
INFO:root:Epoch[92] Time cost=0.219
INFO:root:Epoch[92] Validation-accuracy=0.830833
INFO:root:Epoch[93] Batch [64]	Speed: 63660.03 samples/sec	accuracy=0.852295
INFO:root:Epoch[93] Batch [128]	Speed: 62188.67 samples/sec	accuracy=0.837646
INFO:root:Epoch[93] Batch [192]	Speed: 61139.85 samples/sec	accuracy=0.8

...

INFO:root:Epoch[203] Batch [64]	Speed: 72626.80 samples/sec	accuracy=0.852295
INFO:root:Epoch[203] Batch [128]	Speed: 79855.11 samples/sec	accuracy=0.837402
INFO:root:Epoch[203] Batch [192]	Speed: 78371.03 samples/sec	accuracy=0.842773
INFO:root:Epoch[203] Resetting Data Iterator
INFO:root:Epoch[203] Time cost=0.192
INFO:root:Epoch[203] Validation-accuracy=0.830625
INFO:root:Epoch[204] Batch [64]	Speed: 77211.51 samples/sec	accuracy=0.852295
INFO:root:Epoch[204] Batch [128]	Speed: 75032.40 samples/sec	accuracy=0.837402
INFO:root:Epoch[204] Batch [192]	Speed: 79543.43 samples/sec	accuracy=0.842773
INFO:root:Epoch[204] Resetting Data Iterator
INFO:root:Epoch[204] Time cost=0.191
INFO:root:Epoch[204] Validation-accuracy=0.830625
INFO:root:Epoch[205] Batch [64]	Speed: 80062.77 samples/sec	accuracy=0.852295
INFO:root:Epoch[205] Batch [128]	Speed: 64710.77 samples/sec	accuracy=0.837402
INFO:root:Epoch[205] Batch [192]	Speed: 70160.86 samples/sec	accuracy=0.842773
INFO:root:Epoch[205] Resetti

INFO:root:Epoch[225] Batch [192]	Speed: 69254.12 samples/sec	accuracy=0.842773
INFO:root:Epoch[225] Resetting Data Iterator
INFO:root:Epoch[225] Time cost=0.205
INFO:root:Epoch[225] Validation-accuracy=0.830625
INFO:root:Epoch[226] Batch [64]	Speed: 75448.92 samples/sec	accuracy=0.852295
INFO:root:Epoch[226] Batch [128]	Speed: 73008.27 samples/sec	accuracy=0.837402
INFO:root:Epoch[226] Batch [192]	Speed: 72561.46 samples/sec	accuracy=0.842773
INFO:root:Epoch[226] Resetting Data Iterator
INFO:root:Epoch[226] Time cost=0.205
INFO:root:Epoch[226] Validation-accuracy=0.830625
INFO:root:Epoch[227] Batch [64]	Speed: 67437.89 samples/sec	accuracy=0.852295
INFO:root:Epoch[227] Batch [128]	Speed: 62397.83 samples/sec	accuracy=0.837402
INFO:root:Epoch[227] Batch [192]	Speed: 64401.49 samples/sec	accuracy=0.842773
INFO:root:Epoch[227] Resetting Data Iterator
INFO:root:Epoch[227] Time cost=0.226
INFO:root:Epoch[227] Validation-accuracy=0.830625
INFO:root:Epoch[228] Batch [64]	Speed: 70281.12 sampl

INFO:root:Epoch[247] Time cost=0.214
INFO:root:Epoch[247] Validation-accuracy=0.830625
INFO:root:Epoch[248] Batch [64]	Speed: 77352.65 samples/sec	accuracy=0.852295
INFO:root:Epoch[248] Batch [128]	Speed: 65552.00 samples/sec	accuracy=0.837402
INFO:root:Epoch[248] Batch [192]	Speed: 71825.20 samples/sec	accuracy=0.842773
INFO:root:Epoch[248] Resetting Data Iterator
INFO:root:Epoch[248] Time cost=0.210
INFO:root:Epoch[248] Validation-accuracy=0.830625
INFO:root:Epoch[249] Batch [64]	Speed: 73256.39 samples/sec	accuracy=0.852295
INFO:root:Epoch[249] Batch [128]	Speed: 68055.53 samples/sec	accuracy=0.837402
INFO:root:Epoch[249] Batch [192]	Speed: 81102.54 samples/sec	accuracy=0.842773
INFO:root:Epoch[249] Resetting Data Iterator
INFO:root:Epoch[249] Time cost=0.198
INFO:root:Epoch[249] Validation-accuracy=0.830625
INFO:root:Epoch[250] Batch [64]	Speed: 69532.17 samples/sec	accuracy=0.852295
INFO:root:Epoch[250] Batch [128]	Speed: 72465.97 samples/sec	accuracy=0.837402
INFO:root:Epoch[250]

INFO:root:Epoch[270] Batch [128]	Speed: 63680.56 samples/sec	accuracy=0.837402
INFO:root:Epoch[270] Batch [192]	Speed: 63864.50 samples/sec	accuracy=0.842773
INFO:root:Epoch[270] Resetting Data Iterator
INFO:root:Epoch[270] Time cost=0.246
INFO:root:Epoch[270] Validation-accuracy=0.830625
INFO:root:Epoch[271] Batch [64]	Speed: 67705.25 samples/sec	accuracy=0.852295
INFO:root:Epoch[271] Batch [128]	Speed: 66169.31 samples/sec	accuracy=0.837402
INFO:root:Epoch[271] Batch [192]	Speed: 69220.91 samples/sec	accuracy=0.842773
INFO:root:Epoch[271] Resetting Data Iterator
INFO:root:Epoch[271] Time cost=0.217
INFO:root:Epoch[271] Validation-accuracy=0.830625
INFO:root:Epoch[272] Batch [64]	Speed: 64583.55 samples/sec	accuracy=0.852295
INFO:root:Epoch[272] Batch [128]	Speed: 62096.96 samples/sec	accuracy=0.837402
INFO:root:Epoch[272] Batch [192]	Speed: 61394.17 samples/sec	accuracy=0.842773
INFO:root:Epoch[272] Resetting Data Iterator
INFO:root:Epoch[272] Time cost=0.235
INFO:root:Epoch[272] Val

INFO:root:Epoch[292] Resetting Data Iterator
INFO:root:Epoch[292] Time cost=0.226
INFO:root:Epoch[292] Validation-accuracy=0.830625
INFO:root:Epoch[293] Batch [64]	Speed: 71638.61 samples/sec	accuracy=0.852295
INFO:root:Epoch[293] Batch [128]	Speed: 60106.46 samples/sec	accuracy=0.837402
INFO:root:Epoch[293] Batch [192]	Speed: 63629.85 samples/sec	accuracy=0.842773
INFO:root:Epoch[293] Resetting Data Iterator
INFO:root:Epoch[293] Time cost=0.224
INFO:root:Epoch[293] Validation-accuracy=0.830625
INFO:root:Epoch[294] Batch [64]	Speed: 68092.48 samples/sec	accuracy=0.852295
INFO:root:Epoch[294] Batch [128]	Speed: 73958.39 samples/sec	accuracy=0.837402
INFO:root:Epoch[294] Batch [192]	Speed: 65504.52 samples/sec	accuracy=0.842773
INFO:root:Epoch[294] Resetting Data Iterator
INFO:root:Epoch[294] Time cost=0.210
INFO:root:Epoch[294] Validation-accuracy=0.830625
INFO:root:Epoch[295] Batch [64]	Speed: 82970.09 samples/sec	accuracy=0.852295
INFO:root:Epoch[295] Batch [128]	Speed: 72294.59 sampl

In [12]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'SVM'
epoch = 300
mod.save(prefix, epoch)

INFO:root:Saved checkpoint to "SVM-0300.params"


In [13]:
modelMetric(mod, 'Finish predict of one Layer SVM Network...', False)

INFO:root:Finish predict of one Layer SVM Network...
INFO:root:Test accuracy = 0.830264


             precision    recall  f1-score   support

        0.0       0.92      0.75      0.83      2626
        1.0       0.75      0.92      0.83      2152

avg / total       0.85      0.83      0.83      4778

[[1979  647]
 [ 164 1988]]


## 2.2 DNN + SVM

In [14]:
# DNN + SVM 
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=512)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")

fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=2)

dnnsvm = mx.symbol.SVMOutput(data=fc3, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart.format = 'png'
network_chart.render('DNN + SVM')

'DNN + SVM.png'

<img src='DNN + SVM.png'>

In [15]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = dnnsvm,        
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size), 
    num_epoch = 300,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.001,         # Weight decay for regularization
    },
)
print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 51202.95 samples/sec	accuracy=0.551923
INFO:root:Epoch[0] Batch [128]	Speed: 45058.29 samples/sec	accuracy=0.541260
INFO:root:Epoch[0] Batch [192]	Speed: 43507.99 samples/sec	accuracy=0.561523
INFO:root:Epoch[0] Train-accuracy=0.543347
INFO:root:Epoch[0] Time cost=0.322
INFO:root:Epoch[0] Validation-accuracy=0.549583
INFO:root:Epoch[1] Batch [64]	Speed: 44775.97 samples/sec	accuracy=0.719712
INFO:root:Epoch[1] Batch [128]	Speed: 47095.88 samples/sec	accuracy=0.837158
INFO:root:Epoch[1] Batch [192]	Speed: 51207.68 samples/sec	accuracy=0.833252
INFO:root:Epoch[1] Train-accuracy=0.857863
INFO:root:Epoch[1] Time cost=0.307
INFO:root:Epoch[1] Validation-accuracy=0.847500
INFO:root:Epoch[2] Batch [64]	Speed: 50031.80 samples/sec	accuracy=0.864423
INFO:root:Epoch[2] Batch [128]	Speed: 48331.92 samples/sec	accuracy=0.863037
INFO:root:Epoch[2] Batch [192]	Speed: 51884.74 samples/sec	accuracy=0.856934
INFO:root:Epoch[2] Train-accuracy=0.866935
INFO:root:Epoch

INFO:root:Epoch[22] Validation-accuracy=0.840000
INFO:root:Epoch[23] Batch [64]	Speed: 45130.25 samples/sec	accuracy=0.989663
INFO:root:Epoch[23] Batch [128]	Speed: 46969.05 samples/sec	accuracy=0.993164
INFO:root:Epoch[23] Batch [192]	Speed: 48408.86 samples/sec	accuracy=0.993652
INFO:root:Epoch[23] Train-accuracy=0.989919
INFO:root:Epoch[23] Time cost=0.311
INFO:root:Epoch[23] Validation-accuracy=0.845625
INFO:root:Epoch[24] Batch [64]	Speed: 44377.63 samples/sec	accuracy=0.995673
INFO:root:Epoch[24] Batch [128]	Speed: 48554.16 samples/sec	accuracy=0.994873
INFO:root:Epoch[24] Batch [192]	Speed: 47121.84 samples/sec	accuracy=0.993896
INFO:root:Epoch[24] Train-accuracy=0.991935
INFO:root:Epoch[24] Time cost=0.312
INFO:root:Epoch[24] Validation-accuracy=0.849167
INFO:root:Epoch[25] Batch [64]	Speed: 52148.07 samples/sec	accuracy=0.996394
INFO:root:Epoch[25] Batch [128]	Speed: 48514.53 samples/sec	accuracy=0.995361
INFO:root:Epoch[25] Batch [192]	Speed: 51089.66 samples/sec	accuracy=0.9

INFO:root:Epoch[45] Train-accuracy=0.999496
INFO:root:Epoch[45] Time cost=0.328
INFO:root:Epoch[45] Validation-accuracy=0.845208
INFO:root:Epoch[46] Batch [64]	Speed: 40490.01 samples/sec	accuracy=0.999760
INFO:root:Epoch[46] Batch [128]	Speed: 46292.70 samples/sec	accuracy=0.999268
INFO:root:Epoch[46] Batch [192]	Speed: 47517.66 samples/sec	accuracy=0.999512
INFO:root:Epoch[46] Train-accuracy=0.999496
INFO:root:Epoch[46] Time cost=0.327
INFO:root:Epoch[46] Validation-accuracy=0.845833
INFO:root:Epoch[47] Batch [64]	Speed: 33673.67 samples/sec	accuracy=0.999760
INFO:root:Epoch[47] Batch [128]	Speed: 43838.72 samples/sec	accuracy=0.999512
INFO:root:Epoch[47] Batch [192]	Speed: 43846.44 samples/sec	accuracy=0.999512
INFO:root:Epoch[47] Train-accuracy=0.999496
INFO:root:Epoch[47] Time cost=0.368
INFO:root:Epoch[47] Validation-accuracy=0.844583
INFO:root:Epoch[48] Batch [64]	Speed: 45588.08 samples/sec	accuracy=0.999760
INFO:root:Epoch[48] Batch [128]	Speed: 49427.38 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [192]	Speed: 49399.95 samples/sec	accuracy=0.999756
INFO:root:Epoch[68] Train-accuracy=0.999496
INFO:root:Epoch[68] Time cost=0.298
INFO:root:Epoch[68] Validation-accuracy=0.845417
INFO:root:Epoch[69] Batch [64]	Speed: 41060.19 samples/sec	accuracy=0.999760
INFO:root:Epoch[69] Batch [128]	Speed: 48284.64 samples/sec	accuracy=0.999756
INFO:root:Epoch[69] Batch [192]	Speed: 50296.77 samples/sec	accuracy=0.999756
INFO:root:Epoch[69] Train-accuracy=0.999496
INFO:root:Epoch[69] Time cost=0.311
INFO:root:Epoch[69] Validation-accuracy=0.846458
INFO:root:Epoch[70] Batch [64]	Speed: 51370.14 samples/sec	accuracy=0.999760
INFO:root:Epoch[70] Batch [128]	Speed: 48972.98 samples/sec	accuracy=0.999756
INFO:root:Epoch[70] Batch [192]	Speed: 46907.88 samples/sec	accuracy=0.999756
INFO:root:Epoch[70] Train-accuracy=0.999496
INFO:root:Epoch[70] Time cost=0.301
INFO:root:Epoch[70] Validation-accuracy=0.846667
INFO:root:Epoch[71] Batch [64]	Speed: 45542.64 samples/sec	accuracy=0

INFO:root:Epoch[91] Batch [128]	Speed: 46018.29 samples/sec	accuracy=0.999756
INFO:root:Epoch[91] Batch [192]	Speed: 50569.63 samples/sec	accuracy=1.000000
INFO:root:Epoch[91] Train-accuracy=1.000000
INFO:root:Epoch[91] Time cost=0.313
INFO:root:Epoch[91] Validation-accuracy=0.846875
INFO:root:Epoch[92] Batch [64]	Speed: 45683.24 samples/sec	accuracy=0.999760
INFO:root:Epoch[92] Batch [128]	Speed: 45007.29 samples/sec	accuracy=0.999756
INFO:root:Epoch[92] Batch [192]	Speed: 52586.55 samples/sec	accuracy=1.000000
INFO:root:Epoch[92] Train-accuracy=1.000000
INFO:root:Epoch[92] Time cost=0.303
INFO:root:Epoch[92] Validation-accuracy=0.846458
INFO:root:Epoch[93] Batch [64]	Speed: 38632.14 samples/sec	accuracy=0.999760
INFO:root:Epoch[93] Batch [128]	Speed: 45272.73 samples/sec	accuracy=0.999756
INFO:root:Epoch[93] Batch [192]	Speed: 51048.52 samples/sec	accuracy=1.000000
INFO:root:Epoch[93] Train-accuracy=1.000000
INFO:root:Epoch[93] Time cost=0.324
INFO:root:Epoch[93] Validation-accuracy=

...

INFO:root:Epoch[225] Train-accuracy=1.000000
INFO:root:Epoch[225] Time cost=0.340
INFO:root:Epoch[225] Validation-accuracy=0.845833
INFO:root:Epoch[226] Batch [64]	Speed: 52985.53 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [128]	Speed: 46530.42 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [192]	Speed: 49032.11 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Train-accuracy=1.000000
INFO:root:Epoch[226] Time cost=0.299
INFO:root:Epoch[226] Validation-accuracy=0.845833
INFO:root:Epoch[227] Batch [64]	Speed: 39941.94 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [128]	Speed: 47227.41 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [192]	Speed: 48830.72 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Train-accuracy=1.000000
INFO:root:Epoch[227] Time cost=0.322
INFO:root:Epoch[227] Validation-accuracy=0.845625
INFO:root:Epoch[228] Batch [64]	Speed: 38278.71 samples/sec	accuracy=1.000000
INFO:root:Epoch[228] Batch [128]	Speed: 42565.84 sampl

INFO:root:Epoch[248] Batch [64]	Speed: 49244.89 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Batch [128]	Speed: 40573.01 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Batch [192]	Speed: 40784.23 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Train-accuracy=1.000000
INFO:root:Epoch[248] Time cost=0.341
INFO:root:Epoch[248] Validation-accuracy=0.844375
INFO:root:Epoch[249] Batch [64]	Speed: 43114.59 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Batch [128]	Speed: 49071.88 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Batch [192]	Speed: 53269.59 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Train-accuracy=1.000000
INFO:root:Epoch[249] Time cost=0.301
INFO:root:Epoch[249] Validation-accuracy=0.844167
INFO:root:Epoch[250] Batch [64]	Speed: 48044.02 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Batch [128]	Speed: 47368.96 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Batch [192]	Speed: 51057.17 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Train-a

INFO:root:Epoch[270] Batch [192]	Speed: 53905.41 samples/sec	accuracy=1.000000
INFO:root:Epoch[270] Train-accuracy=1.000000
INFO:root:Epoch[270] Time cost=0.273
INFO:root:Epoch[270] Validation-accuracy=0.845833
INFO:root:Epoch[271] Batch [64]	Speed: 51249.99 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Batch [128]	Speed: 50827.26 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Batch [192]	Speed: 51869.24 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Train-accuracy=1.000000
INFO:root:Epoch[271] Time cost=0.285
INFO:root:Epoch[271] Validation-accuracy=0.845208
INFO:root:Epoch[272] Batch [64]	Speed: 46386.70 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Batch [128]	Speed: 52546.02 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Batch [192]	Speed: 54591.61 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Train-accuracy=1.000000
INFO:root:Epoch[272] Time cost=0.284
INFO:root:Epoch[272] Validation-accuracy=0.844167
INFO:root:Epoch[273] Batch [64]	Speed: 50454.98 sampl

INFO:root:Epoch[292] Time cost=0.310
INFO:root:Epoch[292] Validation-accuracy=0.847292
INFO:root:Epoch[293] Batch [64]	Speed: 49203.85 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Batch [128]	Speed: 51117.78 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Batch [192]	Speed: 53258.86 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Train-accuracy=1.000000
INFO:root:Epoch[293] Time cost=0.288
INFO:root:Epoch[293] Validation-accuracy=0.848542
INFO:root:Epoch[294] Batch [64]	Speed: 49029.87 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Batch [128]	Speed: 48252.50 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Batch [192]	Speed: 52244.81 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Train-accuracy=1.000000
INFO:root:Epoch[294] Time cost=0.293
INFO:root:Epoch[294] Validation-accuracy=0.848542
INFO:root:Epoch[295] Batch [64]	Speed: 49235.58 samples/sec	accuracy=1.000000
INFO:root:Epoch[295] Batch [128]	Speed: 51538.89 samples/sec	accuracy=1.000000
INFO:root:Epoch[295]

Accuracy: 84.60416666666667 %


In [16]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DNN_SVM'
epoch = 300
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DNN_SVM-0300.params"


In [17]:
modelMetric(mod, 'Finish predict of DNN+SVM...')

INFO:root:Finish predict of DNN+SVM...
INFO:root:Test accuracy = 0.846170


             precision    recall  f1-score   support

        0.0       0.87      0.84      0.86      2626
        1.0       0.82      0.85      0.83      2152

avg / total       0.85      0.85      0.85      4778

[[2214  412]
 [ 323 1829]]


## 2.3 Deeper DNN with dropout + SVM

In [18]:
# Deeper DNN with dropout + SVM
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=1024)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 1024)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
drop1 = mx.sym.Dropout(data=act2, p=0.5)

fc3  = mx.symbol.FullyConnected(data = drop1, name = 'fc3', num_hidden = 512)
act3 = mx.symbol.Activation(data = fc3, name='relu3', act_type="relu")
fc4  = mx.symbol.FullyConnected(data = act3, name = 'fc4', num_hidden = 512)
act4 = mx.symbol.Activation(data = fc4, name='relu4', act_type="relu")

drop2 = mx.sym.Dropout(data=act4, p=0.5)

fc5  = mx.symbol.FullyConnected(data = drop2, name='fc5', num_hidden=2)

ddnn = mx.symbol.SVMOutput(data=fc5, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart.format = 'png'
network_chart.render('DDNN + SVM')

'DDNN + SVM.png'

<img src='DDNN + SVM.png'>

In [19]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ddnn,         # Use the network we just defined
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size),  
    num_epoch = 300,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 30670.29 samples/sec	accuracy=0.552885
INFO:root:Epoch[0] Batch [128]	Speed: 32118.21 samples/sec	accuracy=0.541260
INFO:root:Epoch[0] Batch [192]	Speed: 33686.55 samples/sec	accuracy=0.561523
INFO:root:Epoch[0] Train-accuracy=0.543347
INFO:root:Epoch[0] Time cost=0.499
INFO:root:Epoch[0] Validation-accuracy=0.549583
INFO:root:Epoch[1] Batch [64]	Speed: 31193.65 samples/sec	accuracy=0.553606
INFO:root:Epoch[1] Batch [128]	Speed: 33840.82 samples/sec	accuracy=0.541260
INFO:root:Epoch[1] Batch [192]	Speed: 33729.47 samples/sec	accuracy=0.561523
INFO:root:Epoch[1] Train-accuracy=0.543347
INFO:root:Epoch[1] Time cost=0.446
INFO:root:Epoch[1] Validation-accuracy=0.549583
INFO:root:Epoch[2] Batch [64]	Speed: 32864.60 samples/sec	accuracy=0.553606
INFO:root:Epoch[2] Batch [128]	Speed: 33469.12 samples/sec	accuracy=0.541260
INFO:root:Epoch[2] Batch [192]	Speed: 33895.77 samples/sec	accuracy=0.561523
INFO:root:Epoch[2] Train-accuracy=0.543347
INFO:root:Epoch

INFO:root:Epoch[22] Validation-accuracy=0.846875
INFO:root:Epoch[23] Batch [64]	Speed: 33985.36 samples/sec	accuracy=0.891587
INFO:root:Epoch[23] Batch [128]	Speed: 32761.00 samples/sec	accuracy=0.885986
INFO:root:Epoch[23] Batch [192]	Speed: 34171.86 samples/sec	accuracy=0.883057
INFO:root:Epoch[23] Train-accuracy=0.891633
INFO:root:Epoch[23] Time cost=0.430
INFO:root:Epoch[23] Validation-accuracy=0.845625
INFO:root:Epoch[24] Batch [64]	Speed: 36975.46 samples/sec	accuracy=0.901202
INFO:root:Epoch[24] Batch [128]	Speed: 38905.72 samples/sec	accuracy=0.896729
INFO:root:Epoch[24] Batch [192]	Speed: 37702.77 samples/sec	accuracy=0.894775
INFO:root:Epoch[24] Train-accuracy=0.914315
INFO:root:Epoch[24] Time cost=0.408
INFO:root:Epoch[24] Validation-accuracy=0.844583
INFO:root:Epoch[25] Batch [64]	Speed: 24846.58 samples/sec	accuracy=0.914183
INFO:root:Epoch[25] Batch [128]	Speed: 25168.54 samples/sec	accuracy=0.918945
INFO:root:Epoch[25] Batch [192]	Speed: 24846.54 samples/sec	accuracy=0.9

INFO:root:Epoch[45] Train-accuracy=0.990927
INFO:root:Epoch[45] Time cost=0.423
INFO:root:Epoch[45] Validation-accuracy=0.840625
INFO:root:Epoch[46] Batch [64]	Speed: 31492.01 samples/sec	accuracy=0.994231
INFO:root:Epoch[46] Batch [128]	Speed: 33760.03 samples/sec	accuracy=0.993408
INFO:root:Epoch[46] Batch [192]	Speed: 36832.37 samples/sec	accuracy=0.992676
INFO:root:Epoch[46] Train-accuracy=0.990927
INFO:root:Epoch[46] Time cost=0.422
INFO:root:Epoch[46] Validation-accuracy=0.840417
INFO:root:Epoch[47] Batch [64]	Speed: 32555.94 samples/sec	accuracy=0.994231
INFO:root:Epoch[47] Batch [128]	Speed: 36741.23 samples/sec	accuracy=0.993408
INFO:root:Epoch[47] Batch [192]	Speed: 36918.41 samples/sec	accuracy=0.992676
INFO:root:Epoch[47] Train-accuracy=0.990927
INFO:root:Epoch[47] Time cost=0.423
INFO:root:Epoch[47] Validation-accuracy=0.840208
INFO:root:Epoch[48] Batch [64]	Speed: 24104.59 samples/sec	accuracy=0.994471
INFO:root:Epoch[48] Batch [128]	Speed: 23788.84 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [192]	Speed: 32991.70 samples/sec	accuracy=0.993652
INFO:root:Epoch[68] Train-accuracy=0.992440
INFO:root:Epoch[68] Time cost=0.403
INFO:root:Epoch[68] Validation-accuracy=0.840000
INFO:root:Epoch[69] Batch [64]	Speed: 37063.52 samples/sec	accuracy=0.995913
INFO:root:Epoch[69] Batch [128]	Speed: 39308.61 samples/sec	accuracy=0.996094
INFO:root:Epoch[69] Batch [192]	Speed: 36180.18 samples/sec	accuracy=0.993652
INFO:root:Epoch[69] Train-accuracy=0.992440
INFO:root:Epoch[69] Time cost=0.395
INFO:root:Epoch[69] Validation-accuracy=0.839792
INFO:root:Epoch[70] Batch [64]	Speed: 36320.23 samples/sec	accuracy=0.995913
INFO:root:Epoch[70] Batch [128]	Speed: 36899.85 samples/sec	accuracy=0.996338
INFO:root:Epoch[70] Batch [192]	Speed: 36195.35 samples/sec	accuracy=0.993652
INFO:root:Epoch[70] Train-accuracy=0.992440
INFO:root:Epoch[70] Time cost=0.400
INFO:root:Epoch[70] Validation-accuracy=0.839167
INFO:root:Epoch[71] Batch [64]	Speed: 36011.06 samples/sec	accuracy=0

INFO:root:Epoch[91] Batch [128]	Speed: 39439.28 samples/sec	accuracy=0.996826
INFO:root:Epoch[91] Batch [192]	Speed: 37544.98 samples/sec	accuracy=0.996338
INFO:root:Epoch[91] Train-accuracy=0.992944
INFO:root:Epoch[91] Time cost=0.394
INFO:root:Epoch[91] Validation-accuracy=0.839792
INFO:root:Epoch[92] Batch [64]	Speed: 36695.79 samples/sec	accuracy=0.997837
INFO:root:Epoch[92] Batch [128]	Speed: 35069.40 samples/sec	accuracy=0.997314
INFO:root:Epoch[92] Batch [192]	Speed: 32323.18 samples/sec	accuracy=0.996826
INFO:root:Epoch[92] Train-accuracy=0.993448
INFO:root:Epoch[92] Time cost=0.424
INFO:root:Epoch[92] Validation-accuracy=0.841042
INFO:root:Epoch[93] Batch [64]	Speed: 34893.61 samples/sec	accuracy=0.997837
INFO:root:Epoch[93] Batch [128]	Speed: 37412.28 samples/sec	accuracy=0.997314
INFO:root:Epoch[93] Batch [192]	Speed: 36692.66 samples/sec	accuracy=0.997070
INFO:root:Epoch[93] Train-accuracy=0.993952
INFO:root:Epoch[93] Time cost=0.409
INFO:root:Epoch[93] Validation-accuracy=

...

INFO:root:Epoch[225] Train-accuracy=1.000000
INFO:root:Epoch[225] Time cost=0.471
INFO:root:Epoch[225] Validation-accuracy=0.840833
INFO:root:Epoch[226] Batch [64]	Speed: 25130.99 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [128]	Speed: 36030.92 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Batch [192]	Speed: 35876.23 samples/sec	accuracy=1.000000
INFO:root:Epoch[226] Train-accuracy=1.000000
INFO:root:Epoch[226] Time cost=0.456
INFO:root:Epoch[226] Validation-accuracy=0.841250
INFO:root:Epoch[227] Batch [64]	Speed: 27617.22 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [128]	Speed: 31912.14 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Batch [192]	Speed: 25369.76 samples/sec	accuracy=1.000000
INFO:root:Epoch[227] Train-accuracy=1.000000
INFO:root:Epoch[227] Time cost=0.514
INFO:root:Epoch[227] Validation-accuracy=0.841042
INFO:root:Epoch[228] Batch [64]	Speed: 26821.38 samples/sec	accuracy=1.000000
INFO:root:Epoch[228] Batch [128]	Speed: 33431.09 sampl

INFO:root:Epoch[248] Batch [64]	Speed: 34556.02 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Batch [128]	Speed: 32253.16 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Batch [192]	Speed: 34396.47 samples/sec	accuracy=1.000000
INFO:root:Epoch[248] Train-accuracy=1.000000
INFO:root:Epoch[248] Time cost=0.426
INFO:root:Epoch[248] Validation-accuracy=0.841042
INFO:root:Epoch[249] Batch [64]	Speed: 35879.22 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Batch [128]	Speed: 36596.52 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Batch [192]	Speed: 34502.51 samples/sec	accuracy=1.000000
INFO:root:Epoch[249] Train-accuracy=1.000000
INFO:root:Epoch[249] Time cost=0.404
INFO:root:Epoch[249] Validation-accuracy=0.840625
INFO:root:Epoch[250] Batch [64]	Speed: 32047.09 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Batch [128]	Speed: 34749.42 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Batch [192]	Speed: 33310.91 samples/sec	accuracy=1.000000
INFO:root:Epoch[250] Train-a

INFO:root:Epoch[270] Batch [192]	Speed: 37455.02 samples/sec	accuracy=1.000000
INFO:root:Epoch[270] Train-accuracy=1.000000
INFO:root:Epoch[270] Time cost=0.404
INFO:root:Epoch[270] Validation-accuracy=0.841250
INFO:root:Epoch[271] Batch [64]	Speed: 32058.81 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Batch [128]	Speed: 35799.90 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Batch [192]	Speed: 36366.21 samples/sec	accuracy=1.000000
INFO:root:Epoch[271] Train-accuracy=1.000000
INFO:root:Epoch[271] Time cost=0.417
INFO:root:Epoch[271] Validation-accuracy=0.841042
INFO:root:Epoch[272] Batch [64]	Speed: 33320.60 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Batch [128]	Speed: 34128.21 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Batch [192]	Speed: 35997.40 samples/sec	accuracy=1.000000
INFO:root:Epoch[272] Train-accuracy=1.000000
INFO:root:Epoch[272] Time cost=0.419
INFO:root:Epoch[272] Validation-accuracy=0.840625
INFO:root:Epoch[273] Batch [64]	Speed: 31476.09 sampl

INFO:root:Epoch[292] Time cost=0.451
INFO:root:Epoch[292] Validation-accuracy=0.840833
INFO:root:Epoch[293] Batch [64]	Speed: 32858.31 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Batch [128]	Speed: 33093.51 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Batch [192]	Speed: 36827.47 samples/sec	accuracy=1.000000
INFO:root:Epoch[293] Train-accuracy=1.000000
INFO:root:Epoch[293] Time cost=0.428
INFO:root:Epoch[293] Validation-accuracy=0.840625
INFO:root:Epoch[294] Batch [64]	Speed: 23309.24 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Batch [128]	Speed: 22857.64 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Batch [192]	Speed: 35939.27 samples/sec	accuracy=1.000000
INFO:root:Epoch[294] Train-accuracy=1.000000
INFO:root:Epoch[294] Time cost=0.536
INFO:root:Epoch[294] Validation-accuracy=0.840417
INFO:root:Epoch[295] Batch [64]	Speed: 23724.78 samples/sec	accuracy=1.000000
INFO:root:Epoch[295] Batch [128]	Speed: 23704.22 samples/sec	accuracy=1.000000
INFO:root:Epoch[295]

Accuracy: 84.0 %


In [20]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DDNN_SVM'
epoch = 300
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DDNN_SVM-0300.params"


In [21]:
modelMetric(mod, 'Finish predict of DDNN + SVM...')

INFO:root:Finish predict of DDNN + SVM...
INFO:root:Test accuracy = 0.839682


             precision    recall  f1-score   support

        0.0       0.87      0.83      0.85      2626
        1.0       0.80      0.85      0.83      2152

avg / total       0.84      0.84      0.84      4778

[[2181  445]
 [ 321 1831]]


## 2.4 Inception Batach Normalization

In [22]:
# Reference: www.mxnet.io
# Basic Conv + BN + ReLU factory
def ConvFactory(data, num_filter, kernel, stride=(1,1), pad=(0, 0), act_type="relu"):
    conv = mx.symbol.Convolution(data=data, workspace=256,
                                 num_filter=num_filter, kernel=kernel, stride=stride, pad=pad)
    bn = mx.symbol.BatchNorm(data=conv)
    act = mx.symbol.Activation(data = bn, act_type=act_type)
    return act

# A Simple Downsampling Factory
def DownsampleFactory(data, ch_3x3):
    conv = ConvFactory(data=data, kernel=(3, 3), stride=(2, 2), num_filter=ch_3x3, pad=(1, 1))
    pool = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(2, 2),pad=(1,1), pool_type='max')
    concat = mx.symbol.Concat(*[conv, pool])
    return concat

# A Simple module
def SimpleFactory(data, ch_1x1, ch_3x3):
    conv1x1 = ConvFactory(data=data, kernel=(1, 1), pad=(0, 0), num_filter=ch_1x1)
    conv3x3 = ConvFactory(data=data, kernel=(3, 3), pad=(1, 1), num_filter=ch_3x3)
    concat = mx.symbol.Concat(*[conv1x1, conv3x3])
    return concat

def InceptionFactory(data, num_1x1, num_3x3red, num_3x3, num_d5x5red, num_d5x5, pool, proj, name):
    c1x1 = ConvFactoryB(data=data, num_filter=num_1x1, kernel=(1, 1), name=('%s_1x1' % name))
    c3x3r = ConvFactoryB(data=data, num_filter=num_3x3red, kernel=(1, 1), name=('%s_3x3' % name), suffix='_reduce')
    c3x3 = ConvFactoryB(data=c3x3r, num_filter=num_3x3, kernel=(3, 3), pad=(1, 1), name=('%s_3x3' % name))
    cd5x5r = ConvFactoryB(data=data, num_filter=num_d5x5red, kernel=(1, 1), name=('%s_double_3x3' % name), suffix='_reduce')
    cd5x5 = ConvFactoryB(data=cd5x5r, num_filter=num_d5x5, kernel=(5, 5), pad=(2, 2), name=('%s_double_3x3_1' % name))
    pooling = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(1, 1), pad=(1, 1), pool_type=pool, name=('%s_pool_%s_pool' % (pool, name)))
    cproj = ConvFactoryB(data=pooling, num_filter=proj, kernel=(1, 1), name=('%s_proj' %  name))
    concat = mx.symbol.Concat(*[c1x1, c3x3, cd5x5, cproj], name='ch_concat_%s_chconcat' % name)
    return concat

In [23]:
# Inception Batch Normalization Network
#https://arxiv.org/pdf/1502.03167.pdf
data = mx.sym.Variable('data')

conv1 = ConvFactory(data=data, kernel=(3,3), pad=(1,1), num_filter=96, act_type="relu")
in3a = SimpleFactory(conv1, 32, 32)
in3b = SimpleFactory(in3a, 32, 48)
in3c = DownsampleFactory(in3b, 80)
in4a = SimpleFactory(in3c, 112, 48)
in4b = SimpleFactory(in4a, 96, 64)
in4c = SimpleFactory(in4b, 80, 80)
in4d = SimpleFactory(in4c, 48, 96)
in4e = DownsampleFactory(in4d, 96)
in5a = SimpleFactory(in4e, 176, 160)
in5b = SimpleFactory(in5a, 176, 160)
pool = mx.symbol.Pooling(data=in5b, pool_type="avg", kernel=(7,7), name="global_pool")
flatten = mx.symbol.Flatten(data=pool, name="flatten1")
fc = mx.symbol.FullyConnected(data=flatten, num_hidden=62, name="fc1")
ibn = mx.symbol.SVMOutput(data=fc, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart.format = 'png'
network_chart.render('Inception Batch Normalization')

'Inception Batch Normalization.png'

<img src = 'Inception Batch Normalization.png'>

In [26]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ibn,         # Use the network we just defined
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size),  
    num_epoch = 50,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.0001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Train-accuracy=0.527995
INFO:root:Epoch[0] Time cost=1.314
INFO:root:Epoch[0] Validation-accuracy=0.549583
INFO:root:Epoch[1] Batch [64]	Speed: 1188.31 samples/sec	accuracy=0.629808
INFO:root:Epoch[1] Batch [128]	Speed: 1185.87 samples/sec	accuracy=0.648193
INFO:root:Epoch[1] Batch [192]	Speed: 1170.34 samples/sec	accuracy=0.674561
INFO:root:Epoch[1] Train-accuracy=0.723286
INFO:root:Epoch[1] Time cost=12.126
INFO:root:Epoch[1] Validation-accuracy=0.709167
INFO:root:Epoch[2] Batch [64]	Speed: 1183.88 samples/sec	accuracy=0.764183
INFO:root:Epoch[2] Batch [128]	Speed: 1184.15 samples/sec	accuracy=0.767578
INFO:root:Epoch[2] Batch [192]	Speed: 1158.96 samples/sec	accuracy=0.779541
INFO:root:Epoch[2] Train-accuracy=0.808468
INFO:root:Epoch[2] Time cost=12.170
INFO:root:Epoch[2] Validation-accuracy=0.758333
INFO:root:Epoch[3] Batch [64]	Speed: 1150.42 samples/sec	accuracy=0.820673
INFO:root:Epoch[3] Batch [128]	Speed: 1144.60 samples/sec	accuracy=0.827393
INFO:root:Epoch

INFO:root:Epoch[23] Train-accuracy=0.997984
INFO:root:Epoch[23] Time cost=12.082
INFO:root:Epoch[23] Validation-accuracy=0.800000
INFO:root:Epoch[24] Batch [64]	Speed: 1185.74 samples/sec	accuracy=0.999279
INFO:root:Epoch[24] Batch [128]	Speed: 1185.55 samples/sec	accuracy=0.998291
INFO:root:Epoch[24] Batch [192]	Speed: 1184.20 samples/sec	accuracy=0.998779
INFO:root:Epoch[24] Train-accuracy=0.998488
INFO:root:Epoch[24] Time cost=12.065
INFO:root:Epoch[24] Validation-accuracy=0.802500
INFO:root:Epoch[25] Batch [64]	Speed: 1184.17 samples/sec	accuracy=0.999760
INFO:root:Epoch[25] Batch [128]	Speed: 1186.72 samples/sec	accuracy=0.998535
INFO:root:Epoch[25] Batch [192]	Speed: 1179.42 samples/sec	accuracy=0.999023
INFO:root:Epoch[25] Train-accuracy=0.998488
INFO:root:Epoch[25] Time cost=12.088
INFO:root:Epoch[25] Validation-accuracy=0.802083
INFO:root:Epoch[26] Batch [64]	Speed: 1165.43 samples/sec	accuracy=0.999279
INFO:root:Epoch[26] Batch [128]	Speed: 1140.54 samples/sec	accuracy=0.9990

INFO:root:Epoch[46] Train-accuracy=1.000000
INFO:root:Epoch[46] Time cost=14.098
INFO:root:Epoch[46] Validation-accuracy=0.809792
INFO:root:Epoch[47] Batch [64]	Speed: 949.87 samples/sec	accuracy=1.000000
INFO:root:Epoch[47] Batch [128]	Speed: 1007.57 samples/sec	accuracy=1.000000
INFO:root:Epoch[47] Batch [192]	Speed: 1034.78 samples/sec	accuracy=1.000000
INFO:root:Epoch[47] Train-accuracy=1.000000
INFO:root:Epoch[47] Time cost=14.443
INFO:root:Epoch[47] Validation-accuracy=0.809792
INFO:root:Epoch[48] Batch [64]	Speed: 1013.66 samples/sec	accuracy=1.000000
INFO:root:Epoch[48] Batch [128]	Speed: 1038.83 samples/sec	accuracy=1.000000
INFO:root:Epoch[48] Batch [192]	Speed: 1066.09 samples/sec	accuracy=1.000000
INFO:root:Epoch[48] Train-accuracy=1.000000
INFO:root:Epoch[48] Time cost=13.815
INFO:root:Epoch[48] Validation-accuracy=0.809792
INFO:root:Epoch[49] Batch [64]	Speed: 1074.53 samples/sec	accuracy=1.000000
INFO:root:Epoch[49] Batch [128]	Speed: 1068.27 samples/sec	accuracy=1.00000

Accuracy: 80.97916666666667 %


In [27]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'Inception_BN'
epoch = 50
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "Inception_BN-0050.params"


In [28]:
modelMetric(mod, 'Finish predict of Inception BN..')

INFO:root:Finish predict of Inception BN..
INFO:root:Test accuracy = 0.809962


             precision    recall  f1-score   support

        0.0       0.85      0.80      0.82      2626
        1.0       0.77      0.82      0.80      2152

avg / total       0.81      0.81      0.81      4778

[[2100  526]
 [ 382 1770]]
